# Data Sample

In [ ]:
import pandas as pd 
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import shapely.geometry
import numpy as np
import json


pd.set_option('display.max_columns', None)

In [ ]:
res = pd.read_pickle("data/computed/matches_clean_EWT_delay_seq.pkl")
data = res
df = res

In [ ]:
#data = pd.read_pickle("data/computed/matches_EWT_delay_subsampled.pkl")
#stops_sh = gpd.read_file("data/map/2109_STIB_MIVB_Network/ACTU_STOPS.shp")
#lines_sh = gpd.read_file("data/map/2109_STIB_MIVB_Network/ACTU_LINES.shp")
stops = pd.read_csv("data/timetables/gtfs3Sept/stops.txt")


In [ ]:
coord = stops[["stop_id", "stop_lat", "stop_lon"]].drop_duplicates()
coord_name = stops[["stop_name", "stop_lat", "stop_lon"]].drop_duplicates()

In [ ]:
data.route_long_name.unique()

In [ ]:
data.to_csv("data.csv")

In [ ]:
data_sample = data[(data.route_short_name == "1") & (data.stop_name == 'DE BROUCKERE') & (data.direction_id == 1) & (data.date == '06-09-2021')]

In [ ]:
data_sample = data_sample.sort_values(by="theoretical_time")

In [ ]:
data_sample

In [ ]:
data_sample[data_sample.delay == 561]

In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter(data_sample,
        x="real_time_sec",
        y="delay",
        color="delay_label",
    )

In [ ]:
fig

In [ ]:
coord = stops[["stop_id", "stop_lat", "stop_lon"]]

In [ ]:
test = res.groupby(["stop_id"]).agg({"delay" : ["median"]}).reset_index()
test.columns = test.columns.get_level_values(0)
test = test.merge(
    coord,
    "left",
    "stop_id"
    
)
test

In [ ]:
test[test.delay<0]

Type de map 1

In [ ]:
px.set_mapbox_access_token("pk.eyJ1IjoibWpkYW91ZGkiLCJhIjoiY2xibm54OThyMGdyOTNvcnhqeTYyZmRuYiJ9.rfxe3z8triwA5yvV1XZA-A")
fig = px.density_mapbox(test, lat='stop_lat', lon='stop_lon', z='delay', radius=10,
                        center=dict(lat=50.85, lon=4.45), zoom=9,
                        mapbox_style="white-bg")

fig.update_coloraxes(showscale=False)

fig.show()

Type de map 2

In [ ]:
px.set_mapbox_access_token("pk.eyJ1IjoibWpkYW91ZGkiLCJhIjoiY2xibm54OThyMGdyOTNvcnhqeTYyZmRuYiJ9.rfxe3z8triwA5yvV1XZA-A")
fig = px.scatter_mapbox(test, lat="stop_lat", lon="stop_lon", hover_name="stop_id",
                        color="delay", zoom=10)
fig.update_layout(mapbox_style="white-bg")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(showlegend=False)
fig.show()

Type de map 3

In [ ]:
stops_sh

La méthode en bas marche seulement si c'est des polygones !  
--> px.choropleth_mapbox accepts the geometry of a GeoPandas data frame as the input to geojson if the geometry contains polygons.

In [ ]:
fig = px.choropleth_mapbox(stops_sh,
                           geojson=stops_sh.geometry,
                           locations=stops_sh.stop_id,
                           center={"lat": 50.85, "lon": 4.45},
                           mapbox_style="open-street-map",
                           zoom=10)
fig.show()

Tentative dash leaflet - un peu ratée mais je crois comprendre pourquoi. Quand on transforme en geojson, les coordonées n'ont pas un format très standard

In [ ]:
layer = dl.GeoJSON(json.loads(lines_sh.to_json()))
app = JupyterDash()
app.layout = dl.Map(
    [layer, dl.TileLayer()]
)
#app.run_server(mode="inline")

In [ ]:
gs = json.loads(lines_sh.to_json())
fig = go.Figure(
    data=[
        go.Scattermapbox(
            lat=np.array(feature["geometry"]["coordinates"])[:, 1],
            lon=np.array(feature["geometry"]["coordinates"])[:, 0],
            mode="lines",
            line=dict(width=8, color="#F00")
        )
        for feature in gs["features"]
    ]
)

fig.update_layout(
    margin={"r":0,"t":0,"l":0,"b":0},
    mapbox=go.layout.Mapbox(
        style="stamen-terrain", 
        zoom=10, 
        center_lat = 40.5,
        center_lon = -105.08,
    )
)
fig.show()

In [ ]:
gs

## Information par ligne par jour de la semaine

Ici on peut aller par ligne, par type de jour de la semaine, par cluster etc

In [ ]:
sample_7 = res[(res.route_short_name == "7") & (res.date_label == "workdays") & (res.delay_label == "lat") & (res.direction_id == 0)]

In [ ]:
sample_7

En somme, pour avoir l'impact total causé par la ligne

In [ ]:
s_7_s = sample_7[["stop_id", "delay"]].groupby("stop_id", as_index = False).sum("delay").merge(
    coord,
    "left",
    "stop_id"
).merge(sample_7[["stop_id", "stop_name"]], "left", "stop_id")

In [ ]:
s_7_s

In [ ]:
px.set_mapbox_access_token("pk.eyJ1IjoibWpkYW91ZGkiLCJhIjoiY2xibm54OThyMGdyOTNvcnhqeTYyZmRuYiJ9.rfxe3z8triwA5yvV1XZA-A")
fig = px.density_mapbox(s_7_s, lat='stop_lat', lon='stop_lon', z='delay', radius=10,
                        center=dict(lat=50.85, lon=4.45), zoom=10,
                        color_continuous_scale = "portland" , 
                        mapbox_style="light",
                        hover_name="stop_name", hover_data=["delay"])
fig.show()

En médiane, pour avoir l'impact médian par arrêt

In [ ]:
s_7_m = sample_7[["stop_id", "delay"]].groupby("stop_id", as_index = False).median("delay").merge(
    coord,
    "left",
    "stop_id"
)

In [ ]:

px.set_mapbox_access_token("pk.eyJ1IjoibWpkYW91ZGkiLCJhIjoiY2xibm54OThyMGdyOTNvcnhqeTYyZmRuYiJ9.rfxe3z8triwA5yvV1XZA-A")
fig = px.density_mapbox(s_7_m, lat='stop_lat', lon='stop_lon', z='delay', radius=10,
                        center=dict(lat=50.85, lon=4.45), zoom=10,
                        color_continuous_scale = "portland" , 
                        mapbox_style="stamen-terrain")
fig.show()

Observation tram 7: 
   <br><br/> - beaucoup de retard aux alentours de vdk, soit ils impactent la suite (direction heysel); soit les retards sont vers la fin de la ligne (direction vdk)

In [ ]:
res

In [ ]:
# Number of stops
round(res.stop_id.unique().shape[0])

In [ ]:
# Number of lines
round(res.route_short_name.unique().shape[0])

In [ ]:
# On time positionning (idealemnent, rajouter un threshold with thhresh hold)
round((res[res.regularity == 0].delay_label_120.value_counts()/res[res.regularity == 0].shape[0])*100, 2).to_dict()

In [ ]:
_ = res[(res.regularity == 0)][["stop_name", "delay"]].groupby("stop_name", as_index = False).count().sort_values("delay", ascending = True).merge(
    res[(res.regularity == 0) & (res.delay_label == "lat")][["stop_name", "delay"]].groupby("stop_name", as_index = False).count().sort_values("delay", ascending = True),
    "left",
    "stop_name"
)
_["Probs"] = round((_.delay_y/_.delay_x)*100)
_.sort_values("Probs", ascending=False).iloc[0].Probs

In [ ]:
res[
    (res.regularity == 0)
    & (res.delay_label_strict == "lat")
][["stop_name", "route_short_name", "delay"]].groupby(["stop_name", "route_short_name"], as_index=False).count().sort_values("delay", ascending=False).merge(
    res[(res.regularity == 0)][
    ["stop_name", "delay", "route_short_name"]
].groupby(["stop_name", "route_short_name"], as_index=False).count().sort_values("delay", ascending=True),
"left",
["stop_name", "route_short_name"]
)

In [ ]:
def get_probs():
    _ = res[
            (res.regularity == 0)
            & (res.delay_label_strict == "lat")
        ][["stop_name", "route_short_name", "delay"]].groupby(["stop_name", "route_short_name"], as_index=False).count().sort_values("delay", ascending=False).merge(
            res[(res.regularity == 0)][
            ["stop_name", "delay", "route_short_name"]
        ].groupby(["stop_name", "route_short_name"], as_index=False).count().sort_values("delay", ascending=True),
        "left",
        ["stop_name", "route_short_name"]
        )
    _["Probs"] = round((_.delay_x / _.delay_y) * 100)
    return _[_.delay_y>100].sort_values(by="Probs", ascending= False)

get_probs()



In [ ]:
# Number of vehicule
veh = res.trip_id.unique().shape[0]
"number of vehicule", veh


In [ ]:
fig = px.histogram(res, x="delay_label", color="mode")
fig.show()

In [ ]:
mode = "B"
fig = px.histogram(res[(res.delay.quantile(0.9) > res.delay) & (res.delay > res.delay.quantile(0.05)) & (res["mode"] == mode)], x="delay", histnorm='percent')
fig.show()

In [ ]:
res["EWT_label"] = np.where(res["EWT"] >= 1, "irregular", "regular")


In [ ]:
res.to_pickle("data/computed/matches_EWT_delay_reglab.pkl")

In [ ]:
round((res[res.regularity == 1].EWT_label.value_counts()/res[res.regularity == 1].shape[0])*100, 2).to_dict()

In [ ]:
res[res.regularity == 1].groupby(by = ["route_short_name"], as_index = False).apply(lambda x: x["EWT"].median())

In [ ]:
df = res

data = res

In [ ]:
df[(df.stop_name == "DIEWEG") & (df.route_short_name == "97") & (df.date_normalized == "10-09-2021")& (df.direction_id == 1)].sort_values("headway_real", ascending=False).to_csv("test.csv")

In [ ]:
df[(df.route_short_name == "97") & (df.regularity == 1)].sort_values("EWT", ascending=False)

In [ ]:
df[df.regularity == 1][["stop_name","route_short_name", "EWT"]].groupby(by = ["stop_name", "route_short_name"], as_index = False).median().sort_values("EWT", ascending = False)

In [ ]:
EWT_line = df[df.regularity == 1].groupby(by = ["route_short_name"]).apply(lambda x: x["EWT"].median())
EWT_line.name = 'EWT_line'
df = df.merge(
    EWT_line,
    'left',
    'route_short_name'
)
df["EWT_line_label"] = np.where(df["EWT_line"] >= 1, "irregular", "regular")
line_prob= round((df[df.regularity == 1].EWT_line_label.value_counts()/df[df.regularity == 1].shape[0])*100, 2).to_dict()
worst_line = df[(df.regularity == 1) & (df.EWT_line_label == "irregular")].sort_values(by= 'EWT_line', ascending = False).iloc[0].route_short_name

In [ ]:
data[data.regularity == 1][["route_short_name", "EWT"]].groupby(by=["route_short_name"], as_index=False).median().sort_values("EWT", ascending=False).iloc[0]

In [ ]:
data[data.regularity == 0]

In [ ]:
data["hour"] = data[data.regularity == 0].theoretical_time.apply(lambda x:int(x.split(":")[0]))
data["day"] = data[data.regularity == 0].date_normalized.apply(lambda x:int(x.split("-")[0]))

In [ ]:
data["hour"]

In [ ]:
t = data[["day", "hour", "mode", "delay"]].groupby(["day", "hour", "mode"], as_index=False).median()
t = t[t.hour <= 24]

fig = px.line(t, x="hour", y="delay", color='mode')
"""
.update_layout({
    'plot_bgcolor': 'rgba(0,0,0,0)',
    'paper_bgcolor': 'rgba(0,0,0,0)'
})
"""
fig.show()

In [ ]:
t = data[["day", "mode", "delay"]].groupby(["day", "mode"], as_index=False).median()

fig = px.line(t, x="day", y="delay", color='mode').update_layout({
    'plot_bgcolor': 'rgba(0,0,0,0)',
    'paper_bgcolor': 'rgba(0,0,0,0)'
})
fig.show()

In [ ]:
test = data[["day", "hour", "mode", "delay"]].groupby(["day", "hour", "mode"], as_index=False).median()
test = test[test.hour <= 24]
test

In [ ]:
fig = px.density_heatmap(test, x="day", y="hour", z="delay", histfunc="avg", nbinsx=test.day.shape[0], nbinsy=test.hour.shape[0],)
fig.show()

In [ ]:
fig = px.scatter_3d(test, x='day', y='hour', z='delay', color='day', size_max=18)
fig.show()

In [ ]:
res

In [ ]:
t = data[["day", "hour", "mode", "delay"]].groupby(["day", "hour", "mode"], as_index=False).median()
t = t[t.hour <= 24]

fig = px.line(t, x="hour", y="delay", color='mode')

In [ ]:
t

In [ ]:
_ = data[["day", "mode", "delay_label_120"]].groupby(["day", "mode"], as_index=False).apply(lambda x: pd.Series({'OTP':(x[x.delay_label_120 == "nan"].shape[0]/x.shape[0])*100}))
fig = px.line(_, x="day", y="OTP", color = "mode")
fig.show()

In [ ]:
_ = data[data.regularity == 0][["day", "mode", "delay_label_120"]].groupby(["day", "mode"], as_index=False).apply(lambda x: pd.Series({'OTP':(x[x.delay_label_120 == "nan"].shape[0]/x.shape[0])*100}))
fig = px.line(_, x="day", y="OTP", color = "mode")
fig.show()

In [ ]:
_ = data[["hour", "mode", "delay_label_120"]].groupby(["hour", "mode"], as_index=False).apply(lambda x: pd.Series({'OTP':(x[x.delay_label_120 == "nan"].shape[0]/x.shape[0])*100}))
_ = _[_.hour <= 24]
fig = px.line(_, x="hour", y="OTP", color = "mode", labels = dict(hour='Hour of the day (h)', OTP='OTP (%)'))
fig.show()

In [ ]:
_ = data[["hour", "mode", "delay_label_120"]].groupby(["hour", "mode"], as_index=False).apply(lambda x: pd.Series({'OTP':(x[x.delay_label_120 == "nan"].shape[0]/x.shape[0])*100}))

In [ ]:
_

In [ ]:
_.groupby("mode").median("OTP")

In [ ]:
data

In [ ]:
_ = data[data.regularity == 0][["day", "mode", "delay_label_120"]].groupby(["day", "mode"], as_index=False).apply(lambda x: pd.Series({'OTP':(x[x.delay_label_120 == "nan"].shape[0]/x.shape[0])*100}))

In [ ]:
data_line = 

In [ ]:
_ = data[["stop_name", "delay_label_120", "delay"]].groupby(["stop_name"], as_index=False).apply(lambda x: pd.Series({'OTP':(x[x.delay_label_120 == "nan"].shape[0]/x.shape[0])*100,
'median': x.delay.median()}))

In [ ]:
_ = data[["route_short_name", "delay_label_120", "delay"]].groupby(["route_short_name"], as_index=False).apply(lambda x: pd.Series({'OTP':(x[x.delay_label_120 == "nan"].shape[0]/x.shape[0])*100,
'median': x.delay.median()}))

In [ ]:
px.scatter(_,
x= "OTP",
y='median',
color='route_short_name',
text='route_short_name',
trendline = "lowess",
trendline_scope="overall", 
labels=dict(OTP='OTP (%)', median='median delays (sec)')
)

In [ ]:
_ = data[["stop_name", "delay_label_120", "delay"]].groupby(["stop_name"], as_index=False).apply(lambda x: pd.Series({'OTP':(x[x.delay_label_120 == "nan"].shape[0]/x.shape[0])*100,
'median': x.delay.median()}))

In [ ]:
px.scatter(_,
x= "OTP",
y='median',
hover_name='stop_name',
trendline = "lowess",
trendline_scope="overall", 
color="stop_name",
labels=dict(OTP='OTP (%)', median='median delays (sec)')
)

In [ ]:
fs_delay = pd.read_pickle("data/computed/freq_mining.pkl")

In [ ]:
fs_delay

In [ ]:
res

In [ ]:
t

In [ ]:
t = pd.read_pickle("data/computed/lines_growth_EWT_th.pkl")
#t_g = (t[["route_short_name", "stop_name" ,"date_normalized", "trip_headsign", "succession", "stop_lat", "stop_lon","growth_cut", "lab_growth", "delay"]].groupby(["route_short_name", "stop_name", "date_normalized", "trip_headsign", "succession"], as_index = False).median()).groupby(["route_short_name", "date_normalized", "direction_id"])

In [ ]:
t

In [ ]:
t.merge(
    coord_name.drop_duplicates(subset=["stop_name"]),
    "left",
    "stop_name"
).to_pickle("data/computed/lines_growth_EWT_th.pkl")

In [ ]:
t_g.groupby(["route_short_name", "date_normalized", "direction_id"])

In [ ]:
coord

In [ ]:
route = "7"
direction = "VANDERKINDERE"
date = "15-09-2021"

sample_date = t_g[(t_g.route_short_name == route) & (t_g.trip_headsign == direction)& (t_g["date_normalized"] == date)].sort_values("succession")
sample_date

In [ ]:
px.set_mapbox_access_token("pk.eyJ1IjoibWpkYW91ZGkiLCJhIjoiY2xibm54OThyMGdyOTNvcnhqeTYyZmRuYiJ9.rfxe3z8triwA5yvV1XZA-A")
sample_date["lab_growth"] = sample_date["lab_growth"].astype(str) 
fig = px.scatter_mapbox(sample_date, 
    lat="stop_lat", 
    lon="stop_lon", 
    hover_name="stop_name",
    size = "delay",
    color="lab_growth", 
    zoom=10)


fig.update_layout(mapbox_style="light", margin={"r": 0, "t": 0, "l": 0, "b": 0})

In [ ]:
res